### Libraries

In [131]:
#!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

import pandas as pd

import time

In [132]:
pd.__version__

'0.25.2'

# Data import

In [216]:
clean = pd.read_csv("top_2000_clean.csv")

print(len(clean))
clean.head()


4445


,titel,artiest,jaar,id,1999,2000,2001,2002,2003,2004,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Bohemian Rhapsody,Queen,1975,7tFiyTwD0nx5a1eklYtX2J,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0
1,Hotel California,Eagles,1977,40riOy7x9W7GXjyGp4pjAv,2.0,4.0,3.0,3.0,2.0,2.0,...,1.0,2.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,2.0
2,Child In Time,Deep Purple,1972,5TrfusgKC1BYic5LaWCoKm,3.0,2.0,2.0,2.0,3.0,3.0,...,4.0,3.0,4.0,4.0,4.0,8.0,5.0,5.0,6.0,12.0
3,Stairway To Heaven,Led Zeppelin,1971,5CQ30WqJwcep0pYcV4AMNc,4.0,3.0,4.0,4.0,4.0,4.0,...,5.0,5.0,3.0,3.0,3.0,5.0,3.0,3.0,4.0,5.0
4,Paradise By The Dashboard Light,Meat Loaf,1978,2g7gviEeJr6pyxO7G35EWQ,5.0,5.0,5.0,5.0,5.0,6.0,...,9.0,11.0,19.0,22.0,17.0,30.0,27.0,27.0,29.0,35.0


# Spotipy 

In [136]:
import json
with open("login_token.json") as id_file:
    ids = json.load(id_file)
print(ids)

{'client_id': '1a316693f32b4bd4acee870703d5338b', 'client_secret': 'c96700aaecc545289d156723d97f6580'}


In [137]:
# create client id yourself -> https://developer.spotify.com/dashboard/applications
client_id = ids["client_id"]
client_secret = ids["client_secret"]

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [138]:
# track_id = clean["id"][100]
track_ids = clean["id"]

In [139]:
# #test
# name = "queen Bohemian Rapsody"
# result = sp.search(name) #search query

# result = sp.search() #search query
# result#['tracks']['items']#[0]["album"]["uri"]

result = sp.track(track_id) #search query
result["name"]

# result = sp.album('spotify:album:1WAlaVMaCstzAQ1Y5i0VeX')
# result["genres"]

'Waterloo'

In [218]:
# #one wrong uri
# sp.search("Fat Matress Magic Forest")
clean["id"][2208] = "0WyqgyavXfhHZhLODUrjNb"
clean["id"][2208]

/Users/joey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


'0WyqgyavXfhHZhLODUrjNb'

## Extract spotifydata

In [208]:
def Get_track_features(track_ids):
    feature_dict = {}
    #track info
    feature_dict["title_remove_later"] = []
    feature_dict["artist_id"] = []
    feature_dict["album_id"] = []
    feature_dict["image_640"] = []
    feature_dict["image_64"] = []
    feature_dict['popularity'] = []
    feature_dict['duration'] = []
    feature_dict['preview_url'] = []
    
    #features
    feature_dict['acousticness'] = []
    feature_dict['danceability'] = []
    feature_dict['energy'] = []
    feature_dict['instrumentalness'] = []
    feature_dict['liveness'] = []
    feature_dict['loudness'] = []
    feature_dict['speechiness'] = []
    feature_dict['tempo'] = []
    feature_dict['valence'] = []

    #album genre
    feature_dict['genre'] = []    
    
    for i in range(len(track_ids)):
        start = time.time()
        #pull audio features per track
        if track_ids[i] != None:
            #extract track data
            track = sp.track(track_ids[i]) 
            feature_dict["title_remove_later"].append(track["name"])
            artist_id = track["artists"][0]["uri"]
            feature_dict["artist_id"].append(track["artists"][0]["uri"])
            album_id = track["album"]["uri"] #later needed for genre
            feature_dict["album_id"].append(album_id)
            if len(track["album"]["images"]) > 0:
                feature_dict["image_640"].append(track["album"]["images"][0]["url"])
                feature_dict["image_64"].append(track["album"]["images"][2]["url"])  
            else:
                feature_dict["image_640"].append(None)
                feature_dict["image_64"].append(None) 
            feature_dict['popularity'].append(track['popularity'])
            feature_dict['duration'].append(track["duration_ms"])  
            feature_dict['preview_url'].append(track["preview_url"])
            
            #extract feature data
            features = sp.audio_features(track_ids[i])
            feature_dict['acousticness'].append(features[0]['acousticness'])
            feature_dict['danceability'].append(features[0]['danceability'])
            feature_dict['energy'].append(features[0]['energy'])
            feature_dict['instrumentalness'].append(features[0]['instrumentalness'])
            feature_dict['liveness'].append(features[0]['liveness'])
            feature_dict['loudness'].append(features[0]['loudness'])
            feature_dict['speechiness'].append(features[0]['speechiness'])
            feature_dict['tempo'].append(features[0]['tempo'])
            feature_dict['valence'].append(features[0]['valence'])
            
            #extract genre
            album = sp.album(album_id)
            if len(album['genres']) != 0:
                feature_dict['genre'].append(album["genres"]) 
            else: 
                artist = sp.artist(artist_id)
                feature_dict['genre'].append(artist["genres"])
                
        else: #append None if not found in spotify
            feature_dict["artist_id"].append(None)
            feature_dict["album_id"].append(None)
            feature_dict["image_640"].append(None)
            feature_dict["image_64"].append(None)            
            feature_dict['popularity'].append(None)
            feature_dict['duration'].append(None) 
            feature_dict['preview_url'].append(None)
            
            feature_dict['acousticness'].append(None)
            feature_dict['danceability'].append(None)
            feature_dict['energy'].append(None)
            feature_dict['instrumentalness'].append(None)
            feature_dict['liveness'].append(None)
            feature_dict['loudness'].append(None)
            feature_dict['speechiness'].append(None)
            feature_dict['tempo'].append(None)
            feature_dict['valence'].append(None)
            feature_dict['popularity'].append(None)
            
            feature_dict['genre'].append(None)
                
        print(i, "-", i/len(track_ids))
        time.sleep(max(1./5 - (time.time() - start), 0)) # rate limiting 
    return feature_dict

In [220]:
#RUN
feature_dict = Get_track_features(clean["id"])

0 - 0.0
1 - 0.0002249718785151856
2 - 0.0004499437570303712
3 - 0.0006749156355455568
4 - 0.0008998875140607424
5 - 0.0011248593925759281
6 - 0.0013498312710911137
7 - 0.0015748031496062992
8 - 0.0017997750281214847
9 - 0.0020247469066366703
10 - 0.0022497187851518562
11 - 0.0024746906636670418
12 - 0.0026996625421822273
13 - 0.002924634420697413
14 - 0.0031496062992125984
15 - 0.003374578177727784
16 - 0.0035995500562429695
17 - 0.0038245219347581554
18 - 0.0040494938132733405
19 - 0.0042744656917885265
20 - 0.0044994375703037125
21 - 0.004724409448818898
22 - 0.0049493813273340835
23 - 0.005174353205849269
24 - 0.005399325084364455
25 - 0.00562429696287964
26 - 0.005849268841394826
27 - 0.006074240719910012
28 - 0.006299212598425197
29 - 0.006524184476940383
30 - 0.006749156355455568
31 - 0.006974128233970754
32 - 0.007199100112485939
33 - 0.007424071991001125
34 - 0.007649043869516311
35 - 0.007874015748031496
36 - 0.008098987626546681
37 - 0.008323959505061868
38 - 0.00854893138357

309 - 0.06951631046119235
310 - 0.06974128233970754
311 - 0.06996625421822272
312 - 0.0701912260967379
313 - 0.07041619797525309
retrying ...4secs
314 - 0.07064116985376828
315 - 0.07086614173228346
316 - 0.07109111361079865
317 - 0.07131608548931384
318 - 0.07154105736782902
319 - 0.0717660292463442
320 - 0.0719910011248594
321 - 0.07221597300337458
322 - 0.07244094488188976
323 - 0.07266591676040494
324 - 0.07289088863892014
325 - 0.07311586051743532
326 - 0.0733408323959505
327 - 0.0735658042744657
328 - 0.07379077615298088
329 - 0.07401574803149606
330 - 0.07424071991001124
331 - 0.07446569178852644
332 - 0.07469066366704162
333 - 0.0749156355455568
334 - 0.075140607424072
335 - 0.07536557930258718
336 - 0.07559055118110236
337 - 0.07581552305961754
338 - 0.07604049493813274
retrying ...4secs
339 - 0.07626546681664792
340 - 0.0764904386951631
341 - 0.0767154105736783
342 - 0.07694038245219348
343 - 0.07716535433070866
344 - 0.07739032620922384
345 - 0.07761529808773904
346 - 0.0778

620 - 0.13948256467941508
621 - 0.13970753655793025
622 - 0.13993250843644545
623 - 0.14015748031496064
624 - 0.1403824521934758
625 - 0.140607424071991
626 - 0.14083239595050617
627 - 0.14105736782902137
628 - 0.14128233970753656
629 - 0.14150731158605173
630 - 0.14173228346456693
631 - 0.14195725534308212
632 - 0.1421822272215973
633 - 0.14240719910011249
634 - 0.14263217097862768
635 - 0.14285714285714285
636 - 0.14308211473565804
637 - 0.14330708661417324
638 - 0.1435320584926884
639 - 0.1437570303712036
640 - 0.1439820022497188
641 - 0.14420697412823397
642 - 0.14443194600674916
643 - 0.14465691788526433
644 - 0.14488188976377953
645 - 0.14510686164229472
646 - 0.1453318335208099
647 - 0.14555680539932508
648 - 0.14578177727784028
649 - 0.14600674915635545
650 - 0.14623172103487064
651 - 0.14645669291338584
retrying ...1secs
652 - 0.146681664791901
653 - 0.1469066366704162
654 - 0.1471316085489314
655 - 0.14735658042744657
656 - 0.14758155230596176
657 - 0.14780652418447693
658 - 

932 - 0.20967379077615297
933 - 0.20989876265466817
934 - 0.21012373453318337
935 - 0.21034870641169853
936 - 0.21057367829021373
937 - 0.2107986501687289
938 - 0.2110236220472441
939 - 0.2112485939257593
940 - 0.21147356580427445
941 - 0.21169853768278965
942 - 0.21192350956130485
943 - 0.21214848143982
944 - 0.2123734533183352
945 - 0.2125984251968504
946 - 0.21282339707536557
947 - 0.21304836895388077
948 - 0.21327334083239596
949 - 0.21349831271091113
950 - 0.21372328458942633
951 - 0.2139482564679415
952 - 0.2141732283464567
953 - 0.21439820022497189
954 - 0.21462317210348705
955 - 0.21484814398200225
956 - 0.21507311586051744
957 - 0.2152980877390326
958 - 0.2155230596175478
959 - 0.215748031496063
960 - 0.21597300337457817
961 - 0.21619797525309337
962 - 0.21642294713160856
963 - 0.21664791901012373
964 - 0.21687289088863893
965 - 0.2170978627671541
966 - 0.2173228346456693
967 - 0.21754780652418448
968 - 0.21777277840269965
969 - 0.21799775028121485
970 - 0.21822272215973004
97

1237 - 0.2782902137232846
1238 - 0.2785151856017998
1239 - 0.27874015748031494
1240 - 0.27896512935883017
1241 - 0.27919010123734533
1242 - 0.2794150731158605
1243 - 0.2796400449943757
1244 - 0.2798650168728909
1245 - 0.28008998875140606
1246 - 0.2803149606299213
1247 - 0.28053993250843645
1248 - 0.2807649043869516
1249 - 0.28098987626546684
1250 - 0.281214848143982
1251 - 0.2814398200224972
1252 - 0.28166479190101235
1253 - 0.28188976377952757
1254 - 0.28211473565804274
1255 - 0.2823397075365579
1256 - 0.28256467941507313
1257 - 0.2827896512935883
1258 - 0.28301462317210346
1259 - 0.2832395950506187
1260 - 0.28346456692913385
1261 - 0.283689538807649
1262 - 0.28391451068616425
1263 - 0.2841394825646794
retrying ...2secs
1264 - 0.2843644544431946
1265 - 0.2845894263217098
1266 - 0.28481439820022497
1267 - 0.28503937007874014
1268 - 0.28526434195725536
1269 - 0.28548931383577053
1270 - 0.2857142857142857
1271 - 0.2859392575928009
1272 - 0.2861642294713161
1273 - 0.28638920134983126
1274

...

3079 - 0.6926884139482564
3080 - 0.6929133858267716
3081 - 0.6931383577052869
3082 - 0.693363329583802
3083 - 0.6935883014623172
3084 - 0.6938132733408324
3085 - 0.6940382452193475
3086 - 0.6942632170978628
3087 - 0.694488188976378
3088 - 0.6947131608548931
3089 - 0.6949381327334083
3090 - 0.6951631046119235
3091 - 0.6953880764904387
3092 - 0.6956130483689539
3093 - 0.6958380202474691
retrying ...4secs
3094 - 0.6960629921259842
3095 - 0.6962879640044994
3096 - 0.6965129358830147
3097 - 0.6967379077615298
3098 - 0.696962879640045
3099 - 0.6971878515185602
3100 - 0.6974128233970753
3101 - 0.6976377952755906
3102 - 0.6978627671541058
3103 - 0.6980877390326209
3104 - 0.6983127109111361
3105 - 0.6985376827896513
3106 - 0.6987626546681664
3107 - 0.6989876265466817
3108 - 0.6992125984251969
3109 - 0.699437570303712
3110 - 0.6996625421822272
3111 - 0.6998875140607425
3112 - 0.7001124859392576
3113 - 0.7003374578177728
3114 - 0.700562429696288
3115 - 0.7007874015748031
3116 - 0.7010123734533183

3389 - 0.762429696287964
3390 - 0.7626546681664792
3391 - 0.7628796400449944
3392 - 0.7631046119235095
3393 - 0.7633295838020248
3394 - 0.76355455568054
3395 - 0.7637795275590551
3396 - 0.7640044994375703
3397 - 0.7642294713160855
3398 - 0.7644544431946007
3399 - 0.7646794150731159
3400 - 0.7649043869516311
3401 - 0.7651293588301462
3402 - 0.7653543307086614
3403 - 0.7655793025871767
3404 - 0.7658042744656918
3405 - 0.766029246344207
3406 - 0.7662542182227221
3407 - 0.7664791901012373
3408 - 0.7667041619797526
3409 - 0.7669291338582677
3410 - 0.7671541057367829
3411 - 0.7673790776152981
3412 - 0.7676040494938132
3413 - 0.7678290213723284
3414 - 0.7680539932508437
3415 - 0.7682789651293588
3416 - 0.768503937007874
3417 - 0.7687289088863892
3418 - 0.7689538807649043
3419 - 0.7691788526434196
3420 - 0.7694038245219348
3421 - 0.7696287964004499
3422 - 0.7698537682789651
3423 - 0.7700787401574803
retrying ...2secs
3424 - 0.7703037120359955
3425 - 0.7705286839145107
3426 - 0.7707536557930259

3699 - 0.8321709786276715
3700 - 0.8323959505061868
3701 - 0.8326209223847019
3702 - 0.8328458942632171
3703 - 0.8330708661417323
3704 - 0.8332958380202474
3705 - 0.8335208098987626
3706 - 0.8337457817772779
3707 - 0.833970753655793
3708 - 0.8341957255343082
3709 - 0.8344206974128234
3710 - 0.8346456692913385
3711 - 0.8348706411698538
3712 - 0.835095613048369
3713 - 0.8353205849268841
3714 - 0.8355455568053993
3715 - 0.8357705286839145
3716 - 0.8359955005624297
3717 - 0.8362204724409449
3718 - 0.8364454443194601
3719 - 0.8366704161979752
retrying ...2secs
3720 - 0.8368953880764904
3721 - 0.8371203599550057
3722 - 0.8373453318335208
3723 - 0.837570303712036
3724 - 0.8377952755905512
3725 - 0.8380202474690663
3726 - 0.8382452193475816
3727 - 0.8384701912260968
3728 - 0.8386951631046119
3729 - 0.8389201349831271
3730 - 0.8391451068616423
3731 - 0.8393700787401575
3732 - 0.8395950506186727
3733 - 0.8398200224971879
3734 - 0.840044994375703
3735 - 0.8402699662542182
3736 - 0.840494938132733

4009 - 0.9019122609673791
retrying ...3secs
4010 - 0.9021372328458943
4011 - 0.9023622047244094
4012 - 0.9025871766029246
4013 - 0.9028121484814399
4014 - 0.903037120359955
4015 - 0.9032620922384702
4016 - 0.9034870641169854
4017 - 0.9037120359955005
4018 - 0.9039370078740158
4019 - 0.904161979752531
4020 - 0.9043869516310461
4021 - 0.9046119235095613
4022 - 0.9048368953880765
4023 - 0.9050618672665917
4024 - 0.9052868391451069
4025 - 0.905511811023622
4026 - 0.9057367829021372
4027 - 0.9059617547806524
4028 - 0.9061867266591676
4029 - 0.9064116985376828
4030 - 0.906636670416198
4031 - 0.9068616422947131
4032 - 0.9070866141732283
4033 - 0.9073115860517436
4034 - 0.9075365579302587
4035 - 0.9077615298087739
retrying ...4secs
4036 - 0.9079865016872891
4037 - 0.9082114735658042
4038 - 0.9084364454443195
4039 - 0.9086614173228347
4040 - 0.9088863892013498
4041 - 0.909111361079865
4042 - 0.9093363329583802
4043 - 0.9095613048368953
4044 - 0.9097862767154106
4045 - 0.9100112485939258
4046 - 

retrying ...4secs
4319 - 0.9716535433070866
4320 - 0.9718785151856018
4321 - 0.972103487064117
4322 - 0.9723284589426322
4323 - 0.9725534308211473
4324 - 0.9727784026996625
4325 - 0.9730033745781778
4326 - 0.9732283464566929
4327 - 0.9734533183352081
4328 - 0.9736782902137233
4329 - 0.9739032620922384
4330 - 0.9741282339707537
4331 - 0.9743532058492689
4332 - 0.974578177727784
4333 - 0.9748031496062992
4334 - 0.9750281214848144
4335 - 0.9752530933633295
4336 - 0.9754780652418448
4337 - 0.97570303712036
4338 - 0.9759280089988751
4339 - 0.9761529808773903
4340 - 0.9763779527559056
4341 - 0.9766029246344207
4342 - 0.9768278965129359
4343 - 0.9770528683914511
retrying ...4secs
4344 - 0.9772778402699662
4345 - 0.9775028121484814
4346 - 0.9777277840269967
4347 - 0.9779527559055118
4348 - 0.978177727784027
4349 - 0.9784026996625422
4350 - 0.9786276715410573
4351 - 0.9788526434195726
4352 - 0.9790776152980878
4353 - 0.9793025871766029
4354 - 0.9795275590551181
4355 - 0.9797525309336333
4356 - 

In [221]:
features_df = pd.DataFrame(feature_dict)

In [235]:
print(len(features_df))
features_df.head()

4445


,title_remove_later,artist_id,album_id,image_640,image_64,popularity,duration,preview_url,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,Bohemian Rhapsody - 2011 Mix,spotify:artist:1dfeR4HaWDbWqFHLkxsg1d,spotify:album:6X9k3hSsvQck2OfKYdBbXr,https://i.scdn.co/image/ab67616d0000b273ce4f17...,https://i.scdn.co/image/ab67616d00004851ce4f17...,74,354320,None,0.28800,0.392,0.402,0.000000,0.2430,-9.961,0.0536,143.883,0.228,"[glam rock, rock]"
1,Hotel California - 2013 Remaster,spotify:artist:0ECwFtbIWEVNwjlrfc6xoL,spotify:album:2widuo17g5CEC66IbzveRu,https://i.scdn.co/image/ab67616d0000b273463734...,https://i.scdn.co/image/ab67616d00004851463734...,81,391376,https://p.scdn.co/mp3-preview/50e82c99c20ffa42...,0.00574,0.579,0.508,0.000494,0.0575,-9.484,0.0270,147.125,0.609,"[album rock, classic rock, country rock, folk ..."
2,Child in Time,spotify:artist:568ZhdwyaiCyOGJRtNYhWf,spotify:album:3llL1qaL2RvtyQAthAuRFS,https://i.scdn.co/image/ab67616d0000b273c093f4...,https://i.scdn.co/image/ab67616d00004851c093f4...,44,620373,https://p.scdn.co/mp3-preview/9c5b8bf85a3d429a...,0.17000,0.338,0.461,0.564000,0.0927,-10.740,0.0396,126.698,0.351,"[album rock, blues rock, classic rock, hard ro..."
3,Stairway to Heaven - Remaster,spotify:artist:36QJpDe2go2KgaRleHCDTp,spotify:album:44Ig8dzqOkvkGDzaUof9lK,https://i.scdn.co/image/ab67616d0000b273c8a11e...,https://i.scdn.co/image/ab67616d00004851c8a11e...,77,482830,https://p.scdn.co/mp3-preview/8226164717312bc4...,0.58000,0.338,0.340,0.003200,0.1160,-12.049,0.0339,82.433,0.197,"[album rock, classic rock, hard rock, rock]"
4,Paradise By the Dashboard Light,spotify:artist:7dnB1wSxbYa8CejeVg98hz,spotify:album:6mvI80w5r78niBmwtu7RF9,https://i.scdn.co/image/ab67616d0000b2734111af...,https://i.scdn.co/image/ab67616d000048514111af...,65,508333,https://p.scdn.co/mp3-preview/4b9180c5f9f27ff1...,0.21400,0.326,0.872,0.000000,0.4610,-7.602,0.1220,173.381,0.492,"[album rock, mellow gold, rock, soft rock]"


# Combine

In [241]:
all_track_data = pd.concat([clean, features_df], axis=1)
print(len(all_track_data))
all_track_data.head()

4445


,titel,artiest,jaar,id,1999,2000,2001,2002,2003,2004,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,Bohemian Rhapsody,Queen,1975,7tFiyTwD0nx5a1eklYtX2J,1.0,1.0,1.0,1.0,1.0,1.0,...,0.28800,0.392,0.402,0.000000,0.2430,-9.961,0.0536,143.883,0.228,"[glam rock, rock]"
1,Hotel California,Eagles,1977,40riOy7x9W7GXjyGp4pjAv,2.0,4.0,3.0,3.0,2.0,2.0,...,0.00574,0.579,0.508,0.000494,0.0575,-9.484,0.0270,147.125,0.609,"[album rock, classic rock, country rock, folk ..."
2,Child In Time,Deep Purple,1972,5TrfusgKC1BYic5LaWCoKm,3.0,2.0,2.0,2.0,3.0,3.0,...,0.17000,0.338,0.461,0.564000,0.0927,-10.740,0.0396,126.698,0.351,"[album rock, blues rock, classic rock, hard ro..."
3,Stairway To Heaven,Led Zeppelin,1971,5CQ30WqJwcep0pYcV4AMNc,4.0,3.0,4.0,4.0,4.0,4.0,...,0.58000,0.338,0.340,0.003200,0.1160,-12.049,0.0339,82.433,0.197,"[album rock, classic rock, hard rock, rock]"
4,Paradise By The Dashboard Light,Meat Loaf,1978,2g7gviEeJr6pyxO7G35EWQ,5.0,5.0,5.0,5.0,5.0,6.0,...,0.21400,0.326,0.872,0.000000,0.4610,-7.602,0.1220,173.381,0.492,"[album rock, mellow gold, rock, soft rock]"


In [242]:
#order remained the same
all_track_data[["titel", "title_remove_later"]]

,titel,title_remove_later
0,Bohemian Rhapsody,Bohemian Rhapsody - 2011 Mix
1,Hotel California,Hotel California - 2013 Remaster
2,Child In Time,Child in Time
3,Stairway To Heaven,Stairway to Heaven - Remaster
4,Paradise By The Dashboard Light,Paradise By the Dashboard Light
...,...,...
4440,Kissing A Fool,Kissing a Fool
4441,Lay Your Weapons Down,Lay Your Weapons Down
4442,In My Blood,In My Blood
4443,Hold On Tight,Hold On Tight


In [245]:
#removed title (used to check whether order remained the same)
final_data = all_track_data.drop("title_remove_later", axis=1)

In [247]:
final_data.head()

,titel,artiest,jaar,id,1999,2000,2001,2002,2003,2004,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,Bohemian Rhapsody,Queen,1975,7tFiyTwD0nx5a1eklYtX2J,1.0,1.0,1.0,1.0,1.0,1.0,...,0.28800,0.392,0.402,0.000000,0.2430,-9.961,0.0536,143.883,0.228,"[glam rock, rock]"
1,Hotel California,Eagles,1977,40riOy7x9W7GXjyGp4pjAv,2.0,4.0,3.0,3.0,2.0,2.0,...,0.00574,0.579,0.508,0.000494,0.0575,-9.484,0.0270,147.125,0.609,"[album rock, classic rock, country rock, folk ..."
2,Child In Time,Deep Purple,1972,5TrfusgKC1BYic5LaWCoKm,3.0,2.0,2.0,2.0,3.0,3.0,...,0.17000,0.338,0.461,0.564000,0.0927,-10.740,0.0396,126.698,0.351,"[album rock, blues rock, classic rock, hard ro..."
3,Stairway To Heaven,Led Zeppelin,1971,5CQ30WqJwcep0pYcV4AMNc,4.0,3.0,4.0,4.0,4.0,4.0,...,0.58000,0.338,0.340,0.003200,0.1160,-12.049,0.0339,82.433,0.197,"[album rock, classic rock, hard rock, rock]"
4,Paradise By The Dashboard Light,Meat Loaf,1978,2g7gviEeJr6pyxO7G35EWQ,5.0,5.0,5.0,5.0,5.0,6.0,...,0.21400,0.326,0.872,0.000000,0.4610,-7.602,0.1220,173.381,0.492,"[album rock, mellow gold, rock, soft rock]"


In [248]:
final_data.columns

Index(['titel', 'artiest', 'jaar', 'id', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       'artist_id', 'album_id', 'image_640', 'image_64', 'popularity',
       'duration', 'preview_url', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'genre'],
      dtype='object')

In [260]:
final_data = final_data.rename(columns={"titel":"title", "artiest":"artist", "jaar":"year", "id":"track_id"})

In [261]:
type(final_data["genre"][0])

list

In [262]:
# Save files
final_data.to_json('final_data_27_02.json', orient='records')
final_data.to_csv('final_data_27_02.csv', index = False)
final_data.to_pickle('final_data_27_02.pickle')